In [23]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import spatial
from segmentflow import segment, view
from skimage import measure
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Particle to binder ratio

In [5]:
imgs_labeled = segment.load_images(
    Path('../results/IP_01/IP_01_labeled_voxels'), file_suffix='.tif')
segment.calc_voxel_stats(imgs_labeled)

Loading images...
--> Images loaded as 3D array:  (1010, 630, 630)
Calculating voxel statistics...
--> Voxel ratio of binder to particles: 3.847485693234297


In [8]:
n_voxels = imgs_labeled.shape[0] * imgs_labeled.shape[1] * imgs_labeled.shape[2]
print(f'{n_voxels=}')
n_void = np.count_nonzero(imgs_labeled == 0)
print(f'{n_void=}')
n_binder = np.count_nonzero(imgs_labeled == 1)
print(f'{n_binder=}')
n_particles = np.count_nonzero(imgs_labeled > 1)
print(f'{n_particles=}')
n_remainder = n_voxels - n_void - n_binder - n_particles
print(f'{n_remainder=}')
print(f'{n_binder/n_particles=}')
density_binder = 2  # g/cm3
density_particles = 2.65  # g/cm3
mass_ratio = n_binder / n_particles * density_binder / density_particles
print(f'{mass_ratio=}')

n_voxels=400869000
n_void=251483728
n_binder=118568209
n_particles=30817063
n_remainder=0
n_binder/n_particles=3.847485693234297


In [3]:
3.847 * 2 / 2.65

2.9033962264150945

In [35]:
regions_df = pd.DataFrame(measure.regionprops_table(
    imgs_labeled, properties=['label', 'area', 'bbox']))
regions_df = regions_df.rename(columns={'area' : 'volume'})
regions_df

,label,volume,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5
0,1,118568209,0,12,2,1010,624,625
1,6,47,2,270,319,6,274,325
2,9,1,3,396,327,4,397,328
3,10,15,3,291,337,6,293,341
4,13,128,2,250,351,7,258,358
...,...,...,...,...,...,...,...,...
16621,16634,355,999,227,331,1008,237,339
16622,16635,160,1000,245,333,1007,252,341
16623,16636,48,1002,262,349,1007,268,354
16624,16637,58,1002,286,397,1006,296,400


## Mesh particles using Delaunay triangulation

In [38]:
imgs_single_grain = np.zeros_like(imgs_labeled)
imgs_single_grain[imgs_labeled == 6] = 1
# Return nonzero coords in row-major order (row, col, slice)
coords_tuple = np.nonzero(imgs_single_grain)
# Transpose nonzeros coords in xyz order
coords_xyz = np.transpose([coords_tuple[1], coords_tuple[0], coords_tuple[2]])
print(coords_xyz.shape)
tri = spatial.Delaunay(coords_xyz)
type(tri)

(47, 3)


scipy.spatial._qhull.Delaunay

## Simplify mesh by size

 - [ ] load mesh
 - [ ] check number of vertices/tris (np.unique?)
 - [ ] find small & large grain examples
 - [ ] explore different methods of controlling grain size

In [26]:
stl_path = Path('../results/F83_01_bh/F83_01_bh_STLs')
props_df = pd.read_csv(stl_path / 'F83_01_bh_properties.csv')
props_df.loc[props_df['meshed']]
# props_df.columns

,particleID,meshed,n_voxels,centroid,min_slice,max_slice,min_row,max_row,min_col,max_col
0,1,True,125490261,"497, 508, 478",0,996,18,996,5,983
26,788,True,624,"816, 366, 5",2,10,356,377,810,824
47,828,True,449,"147, 286, 6",3,11,277,292,141,154
53,835,True,1371,"879, 544, 5",2,12,526,564,864,891
70,856,True,702,"213, 188, 7",3,14,181,196,206,222
...,...,...,...,...,...,...,...,...,...,...
28736,29716,True,336,"806, 240, 961",957,966,234,248,802,811
28772,29752,True,490,"762, 179, 963",959,968,173,187,755,768
28870,29850,True,353,"900, 399, 967",962,973,396,404,895,907
28920,29900,True,1647,"912, 373, 969",959,977,362,384,904,919


In [25]:
props_df.loc[props_df.particleID == 894].n_voxels

99    2778
Name: n_voxels, dtype: int64

In [27]:
fig, ax = view.plot_stl(stl_path / '00894.stl')

FileNotFoundError: [Errno 2] No such file or directory: '..\\results\\F83_01_bh\\F83_01_bh_STLs\\00894.stl'